In [3]:
import pandas as pd
import tradingview_screener
from tradingview_screener.query import Query
from tradingview_screener import Column


def run_tradingview_scan(
    select_fields: list,
    filter_conditions: list,
    order_by_field: str = "volume",
    ascending: bool = False,
    limit: int = 50
) -> dict:
    """
Dynamically queries TradingView's Screener API to retrieve market data based on user-defined filters.

    Args:
        select_fields (list): Fields to fetch (e.g., ['name', 'close', 'Perf.Y', 'volume']). 
                            Map user requests for price, volume, or performance to relevant fields.
        filter_conditions (list): Conditions for filtering results. Example:
                                - Price above $50 → {"column": "close", "operation": "greater", "value": 50}
                                - NASDAQ stocks → {"column": "exchange", "operation": "in_range", "value": ["NASDAQ"]}
        order_by_field (str): Field to sort by (e.g., 'Perf.Y' for yearly performance).
        ascending (bool): Set False for descending order (e.g., "top performers").
        limit (int): Max results to return.

    Returns:
        dict: JSON results of stocks that match the conditions. Optimized for dynamic screener queries.
    """
    try:
        # Build the query
        query = Query().select(*select_fields).limit(limit).set_markets("america")

        # Apply filters dynamically
        for condition in filter_conditions:
            col_name = condition.get("column")
            operation = condition.get("operation")
            value = condition.get("value")

            col = Column(col_name)
            if operation == "in_range":
                query = query.where(col.isin(value))
            elif operation == "greater":
                query = query.where(col > value)
            elif operation == "less":
                query = query.where(col < value)
            elif operation == "equal":
                query = query.where(col == value)

        # Apply sorting
        query = query.order_by(order_by_field, ascending=ascending)

        # Execute the query
        total_count, results = query.get_scanner_data()

        # Return results
        return {
            "total_count": total_count,
            "results": results.to_dict(orient="records")
        }
    except Exception as e:
        return {"error": f"Failed to run screener query: {str(e)}"}




In [11]:
# Import necessary libraries (if needed)
import pandas as pd

# Define fields to fetch
select_fields = [
    'name', 'close', 'sector', 'earnings_per_share_diluted_ttm'
]

# Define filter conditions
filter_conditions = [
    {"column": "sector", "operation": "equal", "value": "Technology Services"}
]

# Run the screener with sorting by EPS
response = run_tradingview_scan(
    select_fields=select_fields,
    filter_conditions=filter_conditions,
    order_by_field="earnings_per_share_diluted_ttm",  # Sort by EPS
    ascending=False,  # Highest EPS first
    limit=50  # Return top 50 results
)

# Display the results in a dataframe
results_df = pd.DataFrame(response['results'])
display(results_df)


,ticker,name,close,sector,earnings_per_share_diluted_ttm
0,NASDAQ:SMX,SMX,0.3666,Technology Services,60.618700
1,OTC:CNSWF,CNSWF,3128.5800,Technology Services,30.159965
2,OTC:ADYYF,ADYYF,1494.4950,Technology Services,28.389607
3,NASDAQ:META,META,597.3500,Technology Services,21.234500
4,NYSE:FICO,FICO,2039.1050,Technology Services,20.446500
5,NYSE:CACI,CACI,404.5000,Technology Services,20.188800
6,OTC:NAPRF,NAPRF,215.5900,Technology Services,19.545154
7,NASDAQ:NFLX,NFLX,905.4300,Technology Services,17.672000
8,NYSE:MSCI,MSCI,606.7650,Technology Services,15.227400
9,NASDAQ:SNPS,SNPS,491.1350,Technology Services,14.522200


In [10]:
import pandas as pd

# Define fields to fetch (without filtering by sector)
select_fields = [
    'name', 'close', 'sector', 'earnings_per_share_diluted_ttm'
]

# No filter conditions – fetch all stocks
filter_conditions = []

# Run the screener without sector filtering
response = run_tradingview_scan(
    select_fields=select_fields,
    filter_conditions=filter_conditions,
    order_by_field="earnings_per_share_diluted_ttm",  # Sort by EPS
    ascending=False,  # Highest EPS first
    limit=1000  # Fetch a larger dataset to capture diverse sectors
)

# Process results
results_df = pd.DataFrame(response['results'])

# Show unique sectors
unique_sectors = results_df['sector'].unique()
print("Unique Sectors in the Screener Data:")
for sector in unique_sectors:
    print(sector)


Unique Sectors in the Screener Data:
Finance
Commercial Services
Process Industries
Communications
Consumer Durables
Consumer Non-Durables
Transportation
Health Technology
Retail Trade
Consumer Services
Technology Services
Electronic Technology
Miscellaneous
Distribution Services
Producer Manufacturing
Non-Energy Minerals
Health Services
Energy Minerals
Industrial Services
Utilities


In [25]:
import pandas as pd

# Define fields to fetch
select_fields = ['name', 'close', 'earnings_per_share_diluted_ttm']

# Apply EPS filter (between 0 and 2)
filter_conditions = [
    {"column": "earnings_per_share_diluted_ttm", "operation": "greater", "value": 2}
    ]

# Run the screener query
response = run_tradingview_scan(
    select_fields=select_fields,
    filter_conditions=filter_conditions,
    order_by_field="earnings_per_share_diluted_ttm",
    ascending=True,
    limit=50
)

# Display results
results_df = pd.DataFrame(response['results'])
display(results_df)


,ticker,name,close,earnings_per_share_diluted_ttm
0,NYSE:HCXY,HCXY,24.7900,2.001000
1,NYSE:HTGC,HTGC,19.9900,2.001000
2,NYSE:BGR,BGR,12.5450,2.007600
3,OTC:PSMMY,PSMMY,29.9320,2.008200
4,NYSE:LVS,LVS,51.6350,2.012100
5,NASDAQ:FAST,FAST,72.6000,2.012200
6,NYSE:AEM,AEM,78.4050,2.014355
7,OTC:REPYF,REPYF,11.7500,2.015161
8,NYSE:EFT,EFT,13.2800,2.020500
9,OTC:IPXHY,IPXHY,12.6400,2.021300


In [33]:
from tradingview_screener.query import Query
from tradingview_screener import Column
from langchain_core.tools import tool

@tool
def execute_tradingview_query(query_str: str) -> dict:
    """
    Execute a raw TradingView Screener query string directly.

    Args:
        query_str (str): Raw TradingView Query string written in Python that defines screening logic.

    Example Query:
        "Query().select('name', 'close', 'volume')
        .where(
            Column('market_cap_basic').between(1_000_000, 50_000_000),
            Column('volume') > 1_000_000
        )
        .order_by('volume', ascending=False)
        .limit(25)
        .get_scanner_data()"

    Returns:
        dict: JSON results of the stocks that match the conditions.
    """
    try:
        # Safely evaluate the string to execute the query
        local_context = {"Query": Query, "Column": Column}  # Pass Column directly
        query = eval(query_str, {}, local_context)
        
        # Ensure the query returns results
        total_count, results = query.get_scanner_data()

        return {
            "total_count": total_count,
            "results": results.to_dict(orient="records")
        }
    except Exception as e:
        return {"error": f"Failed to execute query: {str(e)}"}


In [38]:
query_str = """
(Query()
 .select('name', 'High.1M', 'High.All', 'earnings_per_share_diluted_yoy_growth_ttm')
 .where(
     Column('name') == 'AAPL'
 )
 .limit(1))
"""
response = execute_tradingview_query.invoke(query_str)
print(response)


{'total_count': 1, 'results': [{'ticker': 'NASDAQ:AAPL', 'name': 'AAPL', 'High.1M': 260.1, 'High.All': 260.1, 'earnings_per_share_diluted_yoy_growth_ttm': -0.9755779238788207}]}
